In [ ]:
# Required imports
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox, filedialog
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone  # Include timezone

# Initialize global variables for API quota and DataFrame
api_requests_used = 0
api_requests_remaining = 0
df_global = pd.DataFrame()  # Initialize df_global here

# [Include your sports list here]
sports = [
    'americanfootball_cfl',
    'americanfootball_ncaaf',
   # 'americanfootball_ncaaf_championship_winner',
    'americanfootball_nfl',
  #  'americanfootball_nfl_preseason',
  #  'americanfootball_nfl_super_bowl_winner',
    'americanfootball_ufl',
    'aussierules_afl',
    'baseball_mlb',
    'baseball_mlb_preseason',
    'baseball_mlb_world_series_winner',
    'baseball_milb',
    'baseball_npb',
    'baseball_kbo',
    'baseball_ncaa',
    'basketball_euroleague',
    'basketball_nba',
    'basketball_nba_championship_winner',
    'basketball_wnba',
    'basketball_ncaab',
    'basketball_ncaab_championship_winner',
    'boxing_boxing',
    'cricket_big_bash',
    'cricket_caribbean_premier_league',
    'cricket_icc_world_cup',
    'cricket_international_t20',
    'cricket_ipl',
    'cricket_odi',
    'cricket_psl',
    'cricket_t20_blast',
    'cricket_test_match',
 #   'golf_masters_tournament_winner',
 #   'golf_pga_championship_winner',
  #  'golf_the_open_championship_winner',
  #  'golf_us_open_winner',
    'icehockey_nhl',
 #   'icehockey_nhl_championship_winner',
    'icehockey_sweden_hockey_league',
    'icehockey_sweden_allsvenskan',
    'lacrosse_pll',
    'mma_mixed_martial_arts',
 #   'politics_us_presidential_election_winner',
    'rugbyleague_nrl',
    'soccer_africa_cup_of_nations',
    'soccer_argentina_primera_division',
    'soccer_australia_aleague',
    'soccer_austria_bundesliga',
    'soccer_belgium_first_div',
    'soccer_brazil_campeonato',
    'soccer_brazil_serie_b',
    'soccer_chile_campeonato',
    'soccer_china_superleague',
    'soccer_denmark_superliga',
    'soccer_efl_champ',
    'soccer_england_efl_cup',
    'soccer_england_league1',
    'soccer_england_league2',
    'soccer_epl',
    'soccer_fa_cup',
 #   'soccer_fifa_world_cup',
  #  'soccer_fifa_world_cup_womens',
  #  'soccer_fifa_world_cup_winner',
    'soccer_finland_veikkausliiga',
    'soccer_france_ligue_one',
    'soccer_france_ligue_two',
    'soccer_germany_bundesliga',
    'soccer_germany_bundesliga2',
    'soccer_germany_liga3',
    'soccer_greece_super_league',
    'soccer_italy_serie_a',
    'soccer_italy_serie_b',
    'soccer_japan_j_league',
    'soccer_korea_kleague1',
    'soccer_league_of_ireland',
    'soccer_mexico_ligamx',
    'soccer_netherlands_eredivisie',
    'soccer_norway_eliteserien',
    'soccer_poland_ekstraklasa',
    'soccer_portugal_primeira_liga',
    'soccer_spain_la_liga',
    'soccer_spain_segunda_division',
    'soccer_spl',
    'soccer_sweden_allsvenskan',
    'soccer_sweden_superettan',
    'soccer_switzerland_superleague',
    'soccer_turkey_super_league',
    'soccer_uefa_europa_conference_league',
    'soccer_uefa_champs_league',
    'soccer_uefa_champs_league_qualification',
    'soccer_uefa_europa_league',
    'soccer_uefa_european_championship',
    'soccer_uefa_euro_qualification',
    'soccer_uefa_nations_league',
    'soccer_conmebol_copa_america',
    'soccer_conmebol_copa_libertadores',
    'soccer_usa_mls',
    'tennis_atp_aus_open_singles',
    'tennis_atp_canadian_open',
    'tennis_atp_china_open',
    'tennis_atp_cincinnati_open',
    'tennis_atp_french_open',
    'tennis_atp_paris_masters',
    'tennis_atp_shanghai_masters',
    'tennis_atp_us_open',
    'tennis_atp_wimbledon',
    'tennis_wta_aus_open_singles',
    'tennis_wta_canadian_open',
    'tennis_wta_china_open',
    'tennis_wta_cincinnati_open',
    'tennis_wta_french_open',
    'tennis_wta_us_open',
    'tennis_wta_wimbledon',
    'tennis_wta_wuhan_open'
]

# Fetch and prepare odds data
def download_odds(api_quota_label):
    global api_requests_used, api_requests_remaining, df_global

    all_matches = []

    # Calculate date range for 30 days
    now = datetime.now(timezone.utc)
    end_time = now + timedelta(days=30)

    # Define all head-to-head market keys
    h2h_market_keys = ['h2h']

    # Fetch data once for all sports and markets
    for sport in sports:
        # Replace with your API endpoint and key
        markets = ','.join(h2h_market_keys)
        url = f"https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey=7fa2183601e4ace7e50d6e5966f7cb6b&regions=us&markets={markets}&oddsFormat=american"
        response = requests.get(url)

        api_requests_used = int(response.headers.get('x-requests-used', '0'))
        api_requests_remaining = int(response.headers.get('x-requests-remaining', '0'))
        api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
        root.update()  # Update the GUI

        if response.status_code != 200:
            print(f"Failed to get data for sport {sport}: {response.status_code}")
            continue

        result = response.json()
        if not isinstance(result, list):
            print(f"Unexpected result format for sport {sport}")
            continue

        for event in result:
            event_time = datetime.fromisoformat(event['commence_time'].replace('Z', '+00:00'))

            if event_time < now or event_time > end_time:
                continue  # Skip events outside the desired date range

            event_time_str = event_time.strftime('%Y-%m-%d %H:%M:%S')
            for bookmaker in event.get('bookmakers', []):
                if bookmaker['title'] != 'Bovada':
                    continue
                for market in bookmaker.get('markets', []):
                    if market['key'] not in h2h_market_keys:
                        continue
                    outcomes = market.get('outcomes', [])
                    if len(outcomes) < 2:
                        continue  # Skip if not enough outcomes

                    # Handle different numbers of outcomes (e.g., draws in soccer)
                    match = {
                        'sport': sport,
                        'event_time': event_time_str,
                        'bookmaker': bookmaker['title'],
                        'team1': outcomes[0]['name'],
                        'odds1': outcomes[0]['price'],
                        'team2': outcomes[1]['name'] if len(outcomes) > 1 else None,
                        'odds2': outcomes[1]['price'] if len(outcomes) > 1 else None,
                        'draw_odds': outcomes[2]['price'] if len(outcomes) > 2 else None
                    }
                    all_matches.append(match)

    df_global = pd.DataFrame(all_matches)
    return df_global

# Function to populate the Tkinter table
def populate_table(tree, df, date_range):
    # Clear existing rows
    for row in tree.get_children():
        tree.delete(row)

    # Filter data based on date range
    now = datetime.now(timezone.utc)
    if date_range == 'next_day':
        end_time = now + timedelta(days=1)
    elif date_range == 'next_7_days':
        end_time = now + timedelta(days=7)
    elif date_range == 'next_month':
        end_time = now + timedelta(days=30)
    else:
        end_time = now + timedelta(days=365)

    df_filtered = df.copy()
    df_filtered['event_time_dt'] = pd.to_datetime(df_filtered['event_time'])
    df_filtered = df_filtered[(df_filtered['event_time_dt'] >= now) & (df_filtered['event_time_dt'] <= end_time)]

    # Insert new rows
    for index, row in df_filtered.iterrows():
        values = (
            row['sport'],
            row['event_time'],
            row['bookmaker'],
            row['team1'],
            row['odds1'],
            row['team2'],
            row['odds2'],
            row['draw_odds']
        )
        tree.insert('', 'end', values=values)

# Function to refresh the data and table
def refresh_table(tree, api_quota_label, date_range_var):
    global df_global
    date_range = date_range_var.get()
    if df_global.empty:
        df_global = download_odds(api_quota_label)
    populate_table(tree, df_global, date_range)

# Function to export data to CSV
def export_to_csv():
    if df_global.empty:
        messagebox.showwarning("No Data", "There is no data to export.")
        return
    file_path = filedialog.asksaveasfilename(defaultextension='.csv', filetypes=[("CSV Files", "*.csv")])
    if file_path:
        try:
            df_global.to_csv(file_path, index=False)
            messagebox.showinfo("Export Successful", f"Data exported to {file_path}")
        except Exception as e:
            messagebox.showerror("Export Failed", f"An error occurred: {e}")

# Tkinter GUI setup
root = tk.Tk()
root.title("Live Sports Odds")

# Top frame for controls
top_frame = ttk.Frame(root)
top_frame.pack(side='top', fill='x')

# Date range selection
date_range_var = tk.StringVar(value='next_day')
date_options = [('Next Day', 'next_day'), ('Next 7 Days', 'next_7_days'), ('Next Month', 'next_month')]

date_label = ttk.Label(top_frame, text="Select Date Range:")
date_label.pack(side='left', padx=5)

for text, value in date_options:
    rb = ttk.Radiobutton(top_frame, text=text, variable=date_range_var, value=value)
    rb.pack(side='left')

# Button to refresh data
btn_refresh = ttk.Button(top_frame, text="Pull Data", command=lambda: refresh_table(tree, api_quota_label, date_range_var))
btn_refresh.pack(side='left', padx=10)

# Button to export data
btn_export = ttk.Button(top_frame, text="Export to CSV", command=export_to_csv)
btn_export.pack(side='left', padx=10)

# Label to display API quota
api_quota_label = ttk.Label(root, text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
api_quota_label.pack(side='bottom')

# Column headers and Treeview
columns = ('Sport', 'Event Time', 'Bookmaker', 'Team 1', 'Odds 1', 'Team 2', 'Odds 2', 'Draw Odds')
tree = ttk.Treeview(root, columns=columns, show='headings')

for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=120)

tree.pack(side='left', fill='both', expand=True)

# Scrollbar
scrollbar = ttk.Scrollbar(root, orient='vertical', command=tree.yview)
scrollbar.pack(side='right', fill='y')
tree.configure(yscrollcommand=scrollbar.set)

# Implement column sorting
def treeview_sort_column(tv, col, reverse):
    data_list = [(tv.set(k, col), k) for k in tv.get_children('')]
    try:
        data_list.sort(key=lambda t: float(t[0]) if t[0] and t[0].replace('.', '', 1).replace('-', '').isdigit() else t[0], reverse=reverse)
    except Exception as e:
        data_list.sort(reverse=reverse)
    # Rearrange items in sorted positions
    for index, (val, k) in enumerate(data_list):
        tv.move(k, '', index)
    # Reverse sort next time
    tv.heading(col, command=lambda: treeview_sort_column(tv, col, not reverse))

# Attach sorting function to column headers
for col in columns:
    tree.heading(col, text=col, command=lambda _col=col: treeview_sort_column(tree, _col, False))

# Tkinter event loop
root.mainloop()

Failed to get data for sport americanfootball_cfl: 422
Failed to get data for sport americanfootball_ncaaf: 422
Failed to get data for sport americanfootball_nfl: 422
Failed to get data for sport americanfootball_ufl: 422
Failed to get data for sport aussierules_afl: 422
Failed to get data for sport baseball_mlb: 422
Failed to get data for sport baseball_mlb_preseason: 422
Failed to get data for sport baseball_mlb_world_series_winner: 422
Failed to get data for sport baseball_milb: 422
Failed to get data for sport baseball_npb: 422
Failed to get data for sport baseball_kbo: 422
Failed to get data for sport baseball_ncaa: 422
Failed to get data for sport basketball_euroleague: 422
Failed to get data for sport basketball_nba: 422
Failed to get data for sport basketball_nba_championship_winner: 422
Failed to get data for sport basketball_wnba: 422
Failed to get data for sport basketball_ncaab: 422
Failed to get data for sport basketball_ncaab_championship_winner: 422
Failed to get data fo